# Imports

In [1]:
# Standard library imports
import os
import math

# Third-party imports
import sqlite3
import numpy as np
import pandas as pd
import torch

# Local imports

# CUDA
print(f"CUDA available: {torch.cuda.is_available()}")
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

CUDA available: True


# Constants

In [2]:
# Directory
CUR_DIR = os.path.dirname(os.path.realpath('__file__'))

# Cytochrome P450 3A4 IDs
CYP3A4_CHEMBL_ID = 'CHEMBL340' # ['CHEMBL2111472', 'CHEMBL2364675', 'CHEMBL4523986']
CHEMBL_DB_PATH = '/data/rbg/users/vincentf/data_uncertainty/chembl_34/chembl_34/chembl_34_sqlite/chembl_34.db'

# ChEMBL Database

In [3]:
# Connect to chembl databse
conn = sqlite3.connect(CHEMBL_DB_PATH, timeout=10)
conn.row_factory = sqlite3.Row
cur = conn.cursor()

In [4]:
# Value types to extract
value_types = ['IC50', 'IC5', 'Log IC50', 'pIC50', 'log(1/IC50)', '-Log IC50(M)', 'Ratio IC50', 'IC50(app)']
placeholders = ','.join(['?'] * len(value_types))

# Query
query = f'''
    SELECT 
        targets.pref_name AS target,
        targets.chembl_id AS target_id,
        assays.chembl_id AS assay_id, 
        compound_structures.canonical_smiles AS smiles,
        activities.standard_type AS type,
        activities.standard_relation AS relation,
        activities.standard_value AS value,
        activities.standard_units AS unit,
        activities.molregno AS molregno,
        assays.doc_id AS doc_id,
        docs.journal AS journal,
        docs.doi AS doi,
        assays.description AS description,
        assays.assay_type AS assay_type, 
        assays.assay_test_type AS assay_test_type, 
        assays.assay_organism AS assay_organism, 
        assays.assay_tissue AS assay_tissue, 
        assays.assay_cell_type AS assay_cell_type, 
        assays.confidence_score AS assay_confidence_score, 
        activities.data_validity_comment AS data_validity_comment
    FROM 
        activities
    JOIN 
        target_dictionary AS targets ON assays.tid = targets.tid
    JOIN 
        assays ON activities.assay_id = assays.assay_id
    JOIN
        compound_structures USING (molregno)
    JOIN 
        docs ON assays.doc_id = docs.doc_id
    WHERE 
        targets.chembl_id = ?
        AND targets.target_type = 'SINGLE PROTEIN'
        AND activities.standard_type IN ({placeholders})
        AND activities.standard_value != 0
        AND activities.standard_value IS NOT NULL
    ORDER BY 
        activities.assay_id, value DESC
'''

# Execute query
cur.execute(query, (CYP3A4_CHEMBL_ID, *value_types))

# Fetch rows from the database
rows = cur.fetchall()

In [5]:
# Save as dataframe
data = [dict(row) for row in rows]
df = pd.DataFrame(data)
df.to_json(f'{CUR_DIR}/data/chembl_data.json', orient='records')

In [6]:
# Example
for key, value in dict(rows[0]).items():
    print(key, ': ', value)

target :  Cytochrome P450 3A4
target_id :  CHEMBL340
assay_id :  CHEMBL883800
smiles :  CC(=O)N1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(Cl)cc4Cl)O3)cc2)CC1
type :  IC50
relation :  =
value :  1260
unit :  nM
molregno :  255904
doc_id :  11347
journal :  J Med Chem
doi :  10.1021/jm00093a015
description :  Inhibition of progesterone 6-beta-hydroxylase in rat hepatic microsomes
assay_type :  A
assay_test_type :  None
assay_organism :  Rattus norvegicus
assay_tissue :  Liver
assay_cell_type :  None
assay_confidence_score :  8
data_validity_comment :  None
